In [2]:
import json

def get_credentials(filepath):
    with open(filepath, 'r') as f:
        credentials = json.load(f)
    return credentials

# Path to your credentials file
credentials_filepath = 'credentials.json'

# Retreieve credentials
credentials = get_credentials(credentials_filepath)

# Access your credentials
username = credentials['username']
password = credentials['password']

In [ ]:
# Step 1: Log-in w/ credentials

# Step 2: Sold Prices
# Step 3: City (Vancouver)